# Secret Santa Bot

In [1]:
from telegram.ext import Updater
from telegram.ext import CommandHandler
import pandas as pd
import numpy as np
import random as rd

updater = Updater(token=) #here your bot token should be mentioned
dispatcher = updater.dispatcher

In [2]:
updater.start_polling() #starting polling requests. Not the best way to work with bots, better use webhook
# you can read about webhook here: https://core.telegram.org/bots/webhooks

In [2]:
# # USE THIS CODE IF YOU STARTING YOUR BOT FOR THE FIRST TIME OR IN CASE YOU WISH TO CLEAR YOUR DATABASE

# # Reset database
# df = pd.DataFrame(data = [], columns = ['chatid', 'chatname', 'userid', 'username', 'linkid', 'magic'])
# df.to_csv("database.csv", index=False)

## /help

In [3]:
def santahelp(bot, update):
    bot.sendMessage(chat_id=update.message.chat_id, text="Use this bot to play Secret Santa game!\n\nAdd this bot to the group chat to link Santas to donees!\nAdd this bot to your personal chat to find out who is your donee!\n\nList of commands for group chat:\n/member - join the game\n/santas - see the list of Santas\n/magic - get a donee\n/reset - start the game again\n\nList of commands for personal chat:\n/mydonee - get the name of donee")
    
santahelp_handler = CommandHandler('help', santahelp)
dispatcher.add_handler(santahelp_handler)

## /start

In [4]:
def start(bot, update):
    bot.sendMessage(chat_id=update.message.chat_id, text="Use /help to find out what is going on!\n\nPlease, contact @vasin_kd if Secret Santa Bot doesnt't work properly")
    
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

## /member

In [6]:
def member(bot, update):
    chatid = update.message.chat_id
    userid = update.message.from_user.id
    chatname = update.message.chat.title
    if update.message.chat.type != "group":
        bot.sendMessage(chat_id=chatid, text="This command is not avaliable in personal chat.")
    else:
        df = pd.read_csv("database.csv", encoding='cp1251')
        mask = df.chatid == chatid
        chatdf = df[mask]
        if sum(chatdf["magic"]) != 0:
            bot.sendMessage(chat_id=chatid, text="Magic has already been here.\n\nIf you want to start again use /reset\nIf you don't know your donee talk to me @SecretClausBot")
        elif userid in chatdf["userid"].values:        
            bot.sendMessage(chat_id=chatid, text="This Santa has already registered!")
        else:
            if update.message.from_user.username:
                username = "@" + update.message.from_user.username
            else:
                username = update.message.from_user.first_name + " " + update.message.from_user.last_name
            number = len(chatdf["userid"].values)
            if number == 0:
                bot.sendMessage(chat_id=update.message.chat_id, text="The first Secret Santa is here!")
            else:
                bot.sendMessage(chat_id=update.message.chat_id, text="One more player with us!\nWow, now there are "+str(number+1)+" Secret Santas!\nIf all Santas are here press /magic")
            newone = pd.DataFrame(data = [[chatid, chatname, userid, username, False]], columns = ['chatid', 'chatname', 'userid', 'username', 'magic'])
            result = pd.concat([df, newone], ignore_index=True)
            result.to_csv("database.csv", index=False)

member_handler = CommandHandler('member', member)
dispatcher.add_handler(member_handler, group=1)

## /magic

In [7]:
def magic(bot, update):
    chatid = update.message.chat_id
    if update.message.chat.type != "group":
        bot.sendMessage(chat_id=chatid, text="This command is not avaliable in personal chat.")
    else:
        df = pd.read_csv("database.csv", encoding='cp1251')
        mask = df.chatid == chatid
        chatdf = df[mask]
        if len(chatdf["userid"]) <2:
            bot.sendMessage(chat_id=chatid, text=str(len(chatdf["userid"]))+" is not enough for doing magic!")
        elif sum(chatdf["magic"]) == 0:
            chatdf['linkid'] = chatdf["userid"]
            dif = [1]
            while sum(dif) != 0:
                rd.shuffle(chatdf["linkid"].values)
                dif = (chatdf["userid"] - chatdf['linkid']) == 0
            bot.sendMessage(chat_id=chatid, text="All Santas now may talk to me @SecretClausBot to get their donee name")
            df.ix[list(chatdf.index.values), "magic"] = True
            df.ix[list(chatdf.index.values), "linkid"] = chatdf.ix[list(chatdf.index.values), "linkid"]
            df.to_csv("database.csv", index=False)
        else:
            bot.sendMessage(chat_id=chatid, text="Magic has already been here.\n\nIf you want to start again use /reset\nIf you don't know your donee talk to me @SecretClausBot")

magic_handler = CommandHandler('magic', magic)
dispatcher.add_handler(magic_handler)

## /reset

In [8]:
def reset(bot, update):
    chatid = update.message.chat_id
    if update.message.chat.type != "group":
        bot.sendMessage(chat_id=chatid, text="This command is not avaliable in personal chat.")
    else:
        df = pd.read_csv("database.csv", encoding='cp1251') 
        mask = df['chatid'] != chatid
        df = df[mask]
        df.to_csv("database.csv", index=False)
        bot.sendMessage(chat_id=chatid, text="Now you may start new Secret Santa game")

reset_handler = CommandHandler('reset', reset)
dispatcher.add_handler(reset_handler)

## /mydonee

In [9]:
def mydonee(bot, update):
    userid = update.message.from_user.id
    if update.message.chat.type == "group":
        bot.sendMessage(chat_id=chatid, text="This command is not avaliable in group chat.")
    else:
        df = pd.read_csv("database.csv", encoding='cp1251')
        mask = df.linkid == userid
        chatdf = df[mask]
        if sum(chatdf["magic"]) == 0:
            bot.sendMessage(chat_id=userid, text="You haven't taken any donees.\n\nStart /magic in a group chat to get a donee")
        else:
            for i in chatdf.index.values:
                bot.sendMessage(chat_id=userid, text="You are the Secret Santa for "+chatdf.ix[i, "username"]+" from "+chatdf.ix[i, "chatname"])
            bot.sendMessage(chat_id=userid, text="Thank you for using Secret Santa Bot!\nPlease, rate and leave your suggestions [here](https://telegram.me/storebot?start=secretclausbot)!", parse_mode="Markdown", disable_web_page_preview=True)

            
mydonee_handler = CommandHandler('mydonee', mydonee)
dispatcher.add_handler(mydonee_handler, group=2)

## /santas

In [10]:
def santas(bot, update):
    chatid = update.message.chat_id
    if update.message.chat.type != "group":
        bot.sendMessage(chat_id=chatid, text="This command is not avaliable in personal chat.")
    else:
        df = pd.read_csv("database.csv", encoding='cp1251')
        mask = df.chatid == chatid
        chatdf = df[mask]
        chatdf.index = np.arange(1, len(chatdf) + 1)
        if chatdf["userid"].empty:
            bot.sendMessage(chat_id=chatid, text="No Secret Santas in this chat")
        else:
            bot.sendMessage(chat_id=chatid, text=str(chatdf.ix[:, "username"])[:-30])
            
santas_handler = CommandHandler('santas', santas)
dispatcher.add_handler(santas_handler, group=2)